# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298450529949                   -0.85    5.1         
  2   -8.300201592042       -2.76       -1.26    1.0   82.4ms
  3   -8.300434594799       -3.63       -1.89    2.6    104ms
  4   -8.300461475927       -4.57       -2.75    2.1   95.7ms
  5   -8.300463718541       -5.65       -3.03    3.0    156ms
  6   -8.300464185599       -6.33       -3.24    9.8    159ms
  7   -8.300464413396       -6.64       -3.39    2.6    103ms
  8   -8.300464539581       -6.90       -3.54    1.6   90.1ms
  9   -8.300464610686       -7.15       -3.73    1.4   87.0ms
 10   -8.300464634388       -7.63       -3.91    1.5    132ms
 11   -8.300464643824       -8.03       -4.38    8.5    227ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67365063195                   -0.70    5.8         
  2   -16.67790118884       -2.37       -1.15    1.0    194ms
  3   -16.67900733145       -2.96       -1.87    3.8    259ms
  4   -16.67917470044       -3.78       -2.55    3.5    266ms
  5   -16.67927879269       -3.98       -3.10    4.6    293ms
  6   -16.67928615318       -5.13       -3.52    2.6    252ms
  7   -16.67928620652       -7.27       -3.84    2.1    220ms
  8   -16.67928622090       -7.84       -4.53    2.2    211ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32533266240                   -0.56    6.6         
  2   -33.33246897358       -2.15       -1.00    1.0    655ms
  3   -33.33407817755       -2.79       -1.74    4.0    832ms
  4   -33.33427591519       -3.70       -2.63    2.8    756ms
  5   -33.33694204405       -2.57       -2.49   11.0    1.22s
  6   -33.33694264587       -6.22       -2.49    1.6    659ms
  7   -33.33694140902   +   -5.91       -2.61    1.1    632ms
  8   -33.33693942761   +   -5.70       -2.88    2.9    660ms
  9   -33.33694153705       -5.68       -3.05    1.6    618ms
 10   -33.33694254567       -6.00       -3.18    1.1    597ms
 11   -33.33694349177       -6.02       -3.49    1.2    597ms
 12   -33.33694376222       -6.57      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298430801560                   -0.85    5.1         
  2   -8.300267907210       -2.74       -1.59    1.0   58.6ms
  3   -8.300437383927       -3.77       -2.72    3.2   85.4ms
  4   -8.300449894307       -4.90       -2.66    2.9   97.4ms
  5   -8.300464470683       -4.84       -3.36    1.0   58.1ms
  6   -8.300464624561       -6.81       -3.97    3.2   86.1ms
  7   -8.300464641993       -7.76       -4.45    2.0    100ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32610725113                   -0.56    7.4         
  2   -33.33122959435       -2.29       -1.28    1.5    605ms
  3   -27.35897707399   +    0.78       -0.68    6.2    1.06s
  4   -33.18236144076        0.77       -1.46    4.8    982ms
  5   -33.29750471169       -0.94       -1.44    2.4    721ms
  6   -33.32561746234       -1.55       -1.93    2.0    689ms
  7   -32.90891373613   +   -0.38       -1.25    4.2    982ms
  8   -33.33416376991       -0.37       -2.28    4.6    930ms
  9   -33.33405213992   +   -3.95       -2.29    1.9    653ms
 10   -33.33513495795       -2.97       -2.39    1.4    591ms
 11   -33.33594749371       -3.09       -2.51    1.5    593ms
 12   -33.33669248249       -3.13      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.